# CSE 881 Project

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def get_yfinance_data(ticker_list : list[str], start_date : str, end_date : str):
    """ Get data from yfinance for a list of tickers.
    
    It includes Open, High, Low, Close, Adj. Close, and Volume.
    
    Args:
        ticker_list : List of ticker symbols
        start_date : Start date of the data
        end_date : End date of the data

    Returns:
        A dict mapping ticker symbols to dataframes containing the data
        fetched from yfinance
    """
    
    data_dict = {}

    for ticker_symbol in ticker_list:
        data = yf.download(ticker_symbol, start=start_date, end=end_date)
        data_dict[ticker_symbol] = data

    return data_dict

In [3]:
# Example tickers and dates
aapl = "AAPL"
msft = "MSFT"
tsla = "TSLA"

tickers = [aapl, msft, tsla]

start_date = "2010-01-01"
end_date = "2023-12-31"

In [4]:
yfinance_data_dict = get_yfinance_data(tickers, start_date, end_date)

# Compute returns and add them as a column to the dataframes
for ticker in yfinance_data_dict:
    yfinance_data_dict[ticker]["Returns"] = yfinance_data_dict[ticker]["Close"].pct_change()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
yfinance_data_dict["AAPL"].head()

,Open,High,Low,Close,Adj Close,Volume,Returns
Date,,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.487534,493729600,NaN
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.498751,601904800,0.001729
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.395380,552160000,-0.015906
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.383555,477131200,-0.001849
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.425994,447610800,0.006648


In [6]:
# TODO: Select and add new features

In [7]:
# TODO: Preprocess the data

In [8]:
# TODO: Implement model prototype

In [9]:
# TODO: Implement Markowitz mean-variance model